## Importing Library

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15, poll_frequency=1)
base_url = 'https://www.airbnb.co.in/s/Bengaluru--India/homes?place_id=ChIJbU60yXAWrjsR4E9-UejD3_g&refinement_paths%5B%5D=%2Fhomes'
driver.get(base_url)
driver.maximize_window()

In [3]:
num_pages = 15
all_data = []

In [4]:
for _ in range(num_pages):
    try:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.lxq01kf")))

        all_items = driver.find_elements(By.CSS_SELECTOR, "div.lxq01kf")
        if not all_items:
            break
        
        for item in all_items:
            name = item.find_element(By.CSS_SELECTOR, "div.t1jojoys").text.strip() if item.find_elements(By.CSS_SELECTOR, "div.t1jojoys") else ''
            description = item.find_element(By.CSS_SELECTOR, "span.t6mzqp7").text.strip() if item.find_elements(By.CSS_SELECTOR, "span.t6mzqp7") else ''
            rating_reviews = item.find_element(By.CSS_SELECTOR, "span.r4a59j5").text.strip() if item.find_elements(By.CSS_SELECTOR, "span.r4a59j5") else '' 

            # Handle "New" or other non-numeric values in rating and reviews
            if "out" in rating_reviews:  
                rating = rating_reviews.split(" out")[0]
                reviews = rating_reviews.split(",")[-1].strip().split(" ")[0]  
            else: 
                rating = '0'
                reviews = '0' 

            price = item.find_element(By.CSS_SELECTOR, "span._11jcbg2").text.strip().replace("₹", "").strip() if item.find_elements(By.CSS_SELECTOR, "span._11jcbg2") else '' 

            place = name.split()[-1] if name else ''

            all_data.append({
                'Name': name,
                'Description': description,
                'Price/Night': price,
                'Rating': rating,
                'Reviews': reviews,
                'Place': place  
            })

        # Handling pagination by clicking the "Next" button
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[aria-label='Next']")))
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            
            if 'disabled' not in next_button.get_attribute('class'):
                next_button.click()
                time.sleep(7) 
            else:
                break 

        except Exception:
            break 

    except Exception:
        break

driver.quit()


In [5]:
df = pd.DataFrame(all_data)
df

,Name,Description,Price/Night,Rating,Reviews,Place
0,Flat in Bengaluru,1BHK in Koramangala - HomeTales 1 by Tipiverse,"2,439",4.71,45,Bengaluru
1,Place to stay in Bengaluru,1 bedroom 10 minutes from center,"1,038",4.87,85,Bengaluru
2,Flat in Bengaluru,Cozy Luxurious 301 Entire Independent 1BHK,"1,253",4.9,20,Bengaluru
3,Place to stay in Bengaluru,Garden facing independent room in prime locality!,"1,197",4.99,79,Bengaluru
4,Apartment in Bengaluru,Compact Terrace Flat in Electronic City - 502,"1,512",0,0,Bengaluru
...,...,...,...,...,...,...
265,Home in Bengaluru,Nice and Cozy - 2 Bedroom house,"2,305",4.93,86,Bengaluru
266,"Home in Byatarayanapura, Bengaluru",Once upon a time in Bengaluru- A Quaint 1BHK,"1,900",4.75,126,Bengaluru
267,Home in Bengaluru,Entire 1 BHK in a beautiful neighbourhood,"1,054",5.0,15,Bengaluru
268,Room in Bengaluru,Studio room at the terrace with good views,"1,137",4.92,39,Bengaluru


In [6]:
df.to_csv('airbnb_Bengaluru_listings.csv', index=False, encoding='utf-8')